La phase de préparation des données regroupe deux volets principaux : la récupération des données et leur traitement. Les données ont été collectées principalement à partir de l'API de la Banque Mondiale, complétées et structurées pour répondre aux exigences du modèle de Solow. Cette étape inclut également un travail de nettoyage approfondi afin de résoudre les problèmes courants tels que les valeurs manquantes, les incohérences ou les doublons. Enfin, des transformations et créations de variables clés ont été réalisées pour faciliter leur interprétation et leur exploitation dans les phases ultérieures du projet.

Cette partie vise ainsi à fournir un jeu de données fiable, homogène et prêt à l'analyse, tout en justifiant les choix méthodologiques adoptés à chaque étape.

I- Installation des packages
Pour effectuer les différentes étapes de récupération, traitement et nettoyage des données, plusieurs bibliothèques Python seront utilisées. Voici les principaux packages nécessaires, accompagnés d'une brève explication de leur rôle dans le projet :

pandas : pour la manipulation, le traitement et la structuration des données tabulaires. Il sera essentiel pour charger, nettoyer et exporter les données dans des formats adaptés.
numpy : pour les opérations mathématiques et statistiques de base sur les données.
requests : pour interagir avec l'API de la Banque Mondiale et effectuer les appels HTTP nécessaires pour récupérer les données.
json : pour traiter les réponses au format JSON issues de l'API.
matplotlib et seaborn : pour visualiser les distributions, identifier les anomalies et générer des graphiques descriptifs pour explorer les données.
os : pour gérer les fichiers locaux et organiser les données sauvegardées.
scipy : pour des fonctions statistiques supplémentaires, notamment pour traiter les valeurs aberrantes ou effectuer des imputations avancées.





II-Construction de base de données

Dans le cadre de notre étude, nous allons récupérer les données suivantes depuis l'API de la Banque Mondiale : la population par tête, le taux d'épargne, le taux démographique, ainsi que les indicateurs suivants :

"GDP (constant 2015 US$)": NY.GDP.MKTP.KD

"Gross Fixed Capital Formation (constant 2015 US$)": NE.GDI.FTOT.KD

"Population ages 15-64, total": SP.POP.1564.TO

"Gross Fixed Capital Formation (% GDP)": NE.GDI.FTOT.ZS

Nous allons créer deux bases de données distinctes : l'une pour les pays de l'OCDE et l'autre pour les pays de l'Union Africaine. La base de données pour les 38 pays de l'OCDE sera nommée ocde_data, tandis que celle pour les 55 pays de l'Union Africaine sera nommée ua_data. Le code ci-dessous permet de récupérer ces deux bases de données.


In [4]:
import requests
import pandas as pd

# Liste des indicateurs avec leurs codes
indicators = {
    "GDP (constant 2015 US$)": "NY.GDP.MKTP.KD",
    "Gross Fixed Capital Formation (constant 2015 US$)": "NE.GDI.FTOT.KD",
    "Population ages 15-64, total": "SP.POP.1564.TO",
    "Gross Fixed Capital Formation (% GDP)": "NE.GDI.FTOT.ZS"
}

# Fonction pour récupérer les données
def fetch_data(indicator, country_code='all'):
    url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/{indicator}?format=json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Transformer les données en DataFrame avec ajout des codes pays
def parse_data(api_response):
    records = []
    if api_response and len(api_response) > 1:
        for entry in api_response[1]:
            country = entry.get("country", {}).get("value")  # Nom du pays
            country_code = entry.get("country", {}).get("id")  # Code du pays
            records.append({
                "Country": country,
                "Country Code": country_code,
                "Year": entry.get("date"),
                "Value": entry.get("value")
            })
    return pd.DataFrame(records)

# Exemple d'utilisation pour récupérer les données de l'indicateur "GDP (constant 2015 US$)"
indicator_code = indicators["GDP (constant 2015 US$)"]
data = fetch_data(indicator_code, country_code='FRA')

# Création du DataFrame
df = parse_data(data)
print(df.head(100))  # Afficher les 5 premières lignes du DataFrame

   Country Country Code  Year         Value
0   France           FR  2023  2.656974e+12
1   France           FR  2022  2.638407e+12
2   France           FR  2021  2.575192e+12
3   France           FR  2020  2.419493e+12
4   France           FR  2019  2.616812e+12
5   France           FR  2018  2.569458e+12
6   France           FR  2017  2.522413e+12
7   France           FR  2016  2.465909e+12
8   France           FR  2015  2.439189e+12
9   France           FR  2014  2.412341e+12
10  France           FR  2013  2.389493e+12
11  France           FR  2012  2.375801e+12
12  France           FR  2011  2.368385e+12
13  France           FR  2010  2.317568e+12
14  France           FR  2009  2.273252e+12
15  France           FR  2008  2.340502e+12
16  France           FR  2007  2.334550e+12
17  France           FR  2006  2.279283e+12
18  France           FR  2005  2.224791e+12
19  France           FR  2004  2.188393e+12
20  France           FR  2003  2.128171e+12
21  France           FR  2002  2